# DS 310 Project 2
### Authors: Sean Rendar & Jess Strait

In [ ]:
nvcfdbcmnv            er  r  

In [ ]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [ ]:
# Import useful packages
import pandas as pd
import numpy as np
import re
import datetime
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.model_selection import RandomizedSearchCV
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor, RandomForestRegressor 
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import SelectKBest, mutual_info_regression

In [ ]:
# Read in training and testing data
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [ ]:
print(list(train.columns))
print(train.dtypes)

['id', 'belongs_to_collection', 'budget', 'genres', 'homepage', 'imdb_id', 'original_language', 'original_title', 'overview', 'popularity', 'poster_path', 'production_companies', 'production_countries', 'release_date', 'runtime', 'spoken_languages', 'status', 'tagline', 'title', 'Keywords', 'cast', 'crew', 'revenue']
id                         int64
belongs_to_collection     object
budget                     int64
genres                    object
homepage                  object
imdb_id                   object
original_language         object
original_title            object
overview                  object
popularity               float64
poster_path               object
production_companies      object
production_countries      object
release_date              object
runtime                  float64
spoken_languages          object
status                    object
tagline                   object
title                     object
Keywords                  object
cast                 

In [ ]:
# Combine train and test for feature engineering
# 0 = train, 1 = test
train['tt'] = 0
test['tt'] = 1
data = train.append(test)
print(data.head())

   id                              belongs_to_collection    budget  \
0   1  [{'id': 313576, 'name': 'Hot Tub Time Machine ...  14000000   
1   2  [{'id': 107674, 'name': 'The Princess Diaries ...  40000000   
2   3                                                NaN   3300000   
3   4                                                NaN   1200000   
4   5                                                NaN         0   

                                              genres  \
0                     [{'id': 35, 'name': 'Comedy'}]   
1  [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...   
2                      [{'id': 18, 'name': 'Drama'}]   
3  [{'id': 53, 'name': 'Thriller'}, {'id': 18, 'n...   
4  [{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...   

                            homepage    imdb_id original_language  \
0                                NaN  tt2637294                en   
1                                NaN  tt0368933                en   
2  http://sonyclassics.com/whiplash

In [ ]:
# Create some binary variables for variables of interest to be made numeric
# Was the original language English?
data['original_language'] = data.original_language.fillna("")
data['orig_eng'] = data.original_language.apply(lambda x: 1 if x == 'en' else 0)
print(sum(data.orig_eng))

6351


In [ ]:
# Was the movie produced in the US?
data['production_countries'] = data.production_countries.fillna("")
data['production_countries'] = data.production_countries.apply(lambda x: re.findall("'name': \'(.+?)\'", x))
data['usa_prod'] = data.production_countries.apply(lambda x: 1 if 'United States of America' in x else 0)
print(sum(data.usa_prod))

5617


In [ ]:
# What was the movie genre?
data['genres'] = data['genres'].fillna('')
data['genres'] = data['genres'].apply(lambda x:re.findall("'name': \'(.+?)\'", x))
genre_dummies = data['genres'].str.join('|').str.get_dummies()
genre_dummies['id'] = data['id']
data = data.merge(genre_dummies, on='id')
print(data.columns)
print(sum(data['Crime']))

Index(['id', 'belongs_to_collection', 'budget', 'genres', 'homepage',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'production_companies',
       'production_countries', 'release_date', 'runtime', 'spoken_languages',
       'status', 'tagline', 'title', 'Keywords', 'cast', 'crew', 'revenue',
       'tt', 'orig_eng', 'usa_prod', 'Action', 'Adventure', 'Animation',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy',
       'Foreign', 'History', 'Horror', 'Music', 'Mystery', 'Romance',
       'Science Fiction', 'TV Movie', 'Thriller', 'War', 'Western'],
      dtype='object')
1084


In [ ]:
# Is the movie part of a collection?
data['belongs_to_collection'] = data['belongs_to_collection'].fillna('')
data['film_collect'] = data.belongs_to_collection.apply(lambda x: 0 if x=='' else 1)
print(sum(data.film_collect))

1481


In [ ]:
# Can we text mine for some popular collections?
data['franchise'] = data.belongs_to_collection.apply(lambda x: 1 if re.search("(?:Avengers|Star Wars|Rocky|James Bond|Harry Potter|Rambo|Lord of the Rings|Star Trek|Hobbit|Twilight|X-Men|Indiana Jones|Transformers|Fast and Furious|Pirates of the Caribbean|Batman|Hunger Games|Mission: Impossible|Jurassic Park|Spider-Man|Iron Man|Bourne)", x) else 0)
print(sum(data.franchise))

134


In [ ]:
# Was English part of the spoken language?
data['spoken_languages'] = data.spoken_languages.fillna("")
data['speak_eng'] = data.spoken_languages.apply(lambda x: 1 if 'en' in x else 0)
print(sum(data.speak_eng))

6465


In [ ]:
# Can we text mine for some famous actors?
data['cast'] = data.cast.fillna("")
data['famous_actor'] = data.cast.apply(lambda x: 1 if re.search("(?:Sylvester Stallone|Johnny Depp|Dwayne Johnson|Tom Hanks|Brad Pitt|Morgan Freeman|Leonardo DiCaprio|Clint Eastwood|Matt Damon|Will Smith|Tom Cruise|Harrison Ford|Robert Downey|Hugh Jackman|Samuel Jackson|Robert DeNiro|Bruce Willis)", x) else 0)
print(sum(data.famous_actor))

611


In [ ]:
# Can we text mine for famous actresses?
data['famous_actress'] = data.cast.apply(lambda x: 1 if re.search("(?:Meryl Streep|Jennifer Lawrence|Anne Hathaway|Emma Watson|Sandra Bullock|Halle Berry|Scarlett Johansson|Julia Roberts|Jennifer Aniston|Nicole Kidman)", x) else 0)
print(sum(data.famous_actress))

285


In [ ]:
# Can we text mine for some famous directors?
data['crew'] = data.crew.fillna("")
data['famous_director'] = data.crew.apply(lambda x: 1 if re.search("(?:Steven Spielberg|Martin Scorsese|Quentin Tarantino|Stanley Kubrick|Tim Burton|Christopher Nolan|James Cameron|David Fincher|Robert Zemeckis)", x) else 0)
print(sum(data.famous_director))

213


In [ ]:
# Can we text mine for some popular production companies?
data['production_companies'] = data.production_companies.fillna("")
data['famous_prod'] = data.production_companies.apply(lambda x: 1 if re.search("(?:Disney|Warner Bros|Paramount|MGM|Twentieth Century Fox|Universal Pictures)", x) else 0)
print(sum(data.famous_prod))

2122


In [ ]:
# Does the movie have a tagline?
data['tagline'] = data['tagline'].fillna("")
data['tag_present'] = data.belongs_to_collection.apply(lambda x: 0 if x=='' else 1)
print(sum(data.tag_present))

1481


In [ ]:
# Does the movie have a production company?
data['production_companies'] = data['production_companies'].fillna("")
data['prod_present'] = data.production_companies.apply(lambda x: 0 if x=='' else 1)
print(sum(data.prod_present))

6984


In [ ]:
# Does the movie have a homepage?
data['homepage'] = data['homepage'].fillna("")
data['home_present'] = data.homepage.apply(lambda x: 0 if x=='' else 1)
print(sum(data.home_present))

2366


In [ ]:
# What was the movie budget?
data['budget'] = data['budget'].fillna('')
data['budget_exists'] = data.budget.apply(lambda x: 0 if x < 1 else 1)
print(sum(data.budget_exists))

# Was the movie budget greater than the average $65 million
data['big_budget'] = data.budget.apply(lambda x:1 if x>65000000 else 0)
print(sum(data.big_budget))

5375
720


In [ ]:
# How long was the runtime?
data['runtime'] = data['runtime'].fillna(114)
data['runtime'] = pd.to_numeric(data['runtime'])
data['longer_than_two_fifteen'] = data.runtime.apply(lambda x: 1 if x>135 else 0)
print(sum(data.longer_than_two_fifteen))

664


In [ ]:
# Mine the release date
data['release_date'] = data['release_date'].fillna('')
data['month'] = pd.DatetimeIndex(data['release_date']).month
data['year'] = pd.DatetimeIndex(data['release_date']).year
data['weekday'] = pd.DatetimeIndex(data['release_date']).dayofweek

# Create summer dummies
data['summer'] = data.month.apply(lambda x: 1 if 5<x<9 else 0)

# Create dummies for some decades
data['before_y2k'] = data.year.apply(lambda x: 1 if x<2000 else 0)
data['before_2010'] = data.year.apply(lambda x: 1 if 1999<x<2010 else 0)
data['since_2010'] = data.year.apply(lambda x: 1 if 2011<x else 0)

In [ ]:
# Cleanup remaining NAs
data['budget'] = data.budget.fillna("")
data['popularity'] = data.popularity.fillna("")
data['revenue'] = data.revenue.fillna("")
data['popularity'] = data['popularity'].astype('int64')

In [ ]:
# Limit data frame to variables of interest
num_data = data[['id', 'budget', 'popularity','orig_eng', 'usa_prod', 'tt', 'franchise', 'Action', 'Adventure', 'Animation',
       'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 'Fantasy', 'tag_present', 'home_present',
       'Foreign', 'History', 'Horror', 'Music', 'Mystery', 'Romance', 'prod_present',
       'Science Fiction', 'TV Movie', 'Thriller', 'War', 'Western', 'film_collect', 'speak_eng', 'famous_actor', 'famous_actress','famous_director', 'famous_prod', 'revenue',
                'budget_exists', 'big_budget', 'longer_than_two_fifteen', 'month', 'year', 'summer', 'weekday', 'before_y2k', 'before_2010', 'since_2010', 'runtime']]

In [ ]:
# Split back into training and testing
train = num_data.loc[num_data['tt'] == 0]
test = num_data.loc[num_data['tt'] == 1]
train = train.drop('tt', 1)
test = test.drop('tt', 1)
train['revenue'] = train['revenue'].astype('int64')
train.applymap(np.isreal)

,id,budget,popularity,orig_eng,usa_prod,franchise,Action,Adventure,Animation,Comedy,...,longer_than_two_fifteen,month,year,summer,weekday,before_y2k,before_2010,since_2010,runtime,prod_num
0,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
2996,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
2997,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
2998,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


In [ ]:
# Prepare data for modeling 
x_train = train.loc[:,train.columns!='revenue']
x_train = x_train.drop('id', 1)
y_train = train['revenue']
x_test = test.loc[:,test.columns!='revenue']
x_test = x_test.drop('id', 1)

In [ ]:
# Normalize the data
x_train = x_train.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))
x_test = x_test.apply(lambda x: (x - np.min(x)) / (np.max(x) - np.min(x)))

In [ ]:
x_test['TV Movie'] = x_test['TV Movie'].fillna(0)
x_test['month'] = x_test['month'].fillna(0)
x_test['year'] = x_test['year'].fillna(0)
x_test['weekday'] = x_test['weekday'].fillna(0)
x_test.isna().sum()

budget                     0
popularity                 0
orig_eng                   0
usa_prod                   0
franchise                  0
Action                     0
Adventure                  0
Animation                  0
Comedy                     0
Crime                      0
Documentary                0
Drama                      0
Family                     0
Fantasy                    0
tag_present                0
home_present               0
Foreign                    0
History                    0
Horror                     0
Music                      0
Mystery                    0
Romance                    0
prod_present               0
Science Fiction            0
TV Movie                   0
Thriller                   0
War                        0
Western                    0
film_collect               0
speak_eng                  0
famous_actor               0
famous_actress             0
famous_director            0
famous_prod                0
budget_exists 

In [ ]:
# Start with decision tree classifier
model = DecisionTreeClassifier()
model.fit(x_train, y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [ ]:
y_pred = model.predict(x_train)

accuracy=accuracy_score(y_train, y_pred)
print('Accuracy on training set: '+str(accuracy))

Accuracy on training set: 1.0


In [ ]:
# Baseline 1 RMSLE: 2.5
test_pred = model.predict(x_test)
print(test_pred)

[ 5300000  1000000  1596687 ... 78874843   572809  3623330]


In [ ]:
# First submission RMSLE: 2.94175

In [ ]:
# Create parameter grid for RandomForest cross-validation
param = { 'max_depth': [5, 10, 15, 20, 25, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [10, 25, 50, 100]}

In [ ]:
# Random forest decision tree
forest = RandomForestRegressor()
rf_random = RandomizedSearchCV(estimator = forest, param_distributions = param, n_iter = 4, cv = 4, verbose=2, random_state=42, n_jobs = -1, return_train_score = True)

rf_random.fit(x_train, y_train)

Fitting 4 folds for each of 4 candidates, totalling 16 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  16 out of  16 | elapsed:    3.9s finished


RandomizedSearchCV(cv=4, error_score=nan,
                   estimator=RandomForestRegressor(bootstrap=True,
                                                   ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features='auto',
                                                   max_leaf_nodes=None,
                                                   max_samples=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   n_estimators=100,
                              

In [ ]:
# View results in order of performance
random_df = pd.DataFrame(rf_random.cv_results_).sort_values('mean_test_score', ascending = False)
random_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,param_min_samples_split,param_min_samples_leaf,param_max_features,param_max_depth,params,...,split3_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,mean_train_score,std_train_score
2,0.240231,0.003902,0.012484,0.001115,50,5,2,sqrt,20,"{'n_estimators': 50, 'min_samples_split': 5, '...",...,0.704604,0.702167,0.035749,1,0.866996,0.861138,0.863663,0.864847,0.864161,0.002115
0,0.576629,0.008529,0.010501,0.000495,50,5,4,auto,10,"{'n_estimators': 50, 'min_samples_split': 5, '...",...,0.694651,0.701833,0.020514,2,0.853191,0.863521,0.855475,0.862032,0.858555,0.004331
3,0.562905,0.007968,0.010007,0.000714,50,10,4,auto,10,"{'n_estimators': 50, 'min_samples_split': 10, ...",...,0.692326,0.694973,0.025375,3,0.842610,0.847819,0.835143,0.853307,0.844720,0.006699
1,0.051858,0.003696,0.004620,0.000807,10,2,4,sqrt,25,"{'n_estimators': 10, 'min_samples_split': 2, '...",...,0.671702,0.673322,0.025693,4,0.792505,0.768648,0.773263,0.790013,0.781107,0.010320


In [ ]:
# Fit best estimator to data
rf_model = rf_random.best_estimator_
rf_model.fit(x_train,y_train)
y_pred = rf_model.predict(x_train)

In [ ]:
test_pred = rf_model.predict(x_test)
print(test_pred)

[58812477.66885509  6566516.23483217 55534352.73071918 ...
 74251068.32273747 81059611.44950631 15466008.73756455]


In [ ]:
# Second submission RMSLE: 2.74767

In [ ]:
# Boosted decision tree
ada = AdaBoostRegressor(n_estimators=100, base_estimator=DecisionTreeRegressor(max_depth=10), learning_rate=1e-3, random_state=42)

ada.fit(x_train,y_train)

y_pred = ada.predict(x_train)

In [ ]:
test_pred = ada.predict(x_test)
print(test_pred)

[ 7922983.2         4208492.96226415 19919542.66666667 ...
 49309719.53488372 26260552.25       10904523.60344828]


In [ ]:
# Third submission RMSLE: 2.62765

In [ ]:
# Create parameter grid for DecisionTreeRegressor
param = { 'max_depth': [5, 10, 15, 20, 25, None],
 'max_features': ['auto', 'sqrt', 'log2'],
 'min_samples_leaf': [1, 2, 3, 4, 5, 6],
 'min_samples_split': [2, 5, 10],
 'min_weight_fraction_leaf': [0.0, 0.1, 0.25, 0.5, 0.75]}
cv = StratifiedKFold(n_splits=5, random_state=0, shuffle=True)

In [ ]:
# Decision tree regressor
tree = DecisionTreeRegressor()
tree_random = RandomizedSearchCV(estimator = tree, param_distributions = param, n_iter = 4, cv = cv, verbose=2, random_state=42, n_jobs = -1, return_train_score = True)

tree_random.fit(x_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


C:\Users\jessl\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    2.5s finished


RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=0, shuffle=True),
                   error_score=nan,
                   estimator=DecisionTreeRegressor(ccp_alpha=0.0,
                                                   criterion='mse',
                                                   max_depth=None,
                                                   max_features=None,
                                                   max_leaf_nodes=None,
                                                   min_impurity_decrease=0.0,
                                                   min_impurity_split=None,
                                                   min_samples_leaf=1,
                                                   min_samples_split=2,
                                                   min_weight_fraction_leaf=0.0,
                                                   presort='deprecated',
                                                   ra...
                                

In [ ]:
tree_df = pd.DataFrame(tree_random.cv_results_).sort_values('mean_test_score', ascending = False)
tree_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_min_weight_fraction_leaf,param_min_samples_split,param_min_samples_leaf,param_max_features,param_max_depth,params,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
2,0.033913,0.003358,0.001902,0.000196,0,5,4,auto,20,"{'min_weight_fraction_leaf': 0.0, 'min_samples...",...,0.521630,0.142883,1,0.880054,0.882227,0.848382,0.895114,0.876217,0.876399,0.015379
0,0.013594,0.001069,0.002195,0.000400,0.1,2,4,auto,25,"{'min_weight_fraction_leaf': 0.1, 'min_samples...",...,0.392091,0.029035,2,0.398781,0.414742,0.413401,0.406383,0.412944,0.409250,0.005981
1,0.006407,0.001854,0.000000,0.000000,0.75,2,2,sqrt,None,"{'min_weight_fraction_leaf': 0.75, 'min_sample...",...,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.004511,0.000426,0.000000,0.000000,0.75,2,3,log2,25,"{'min_weight_fraction_leaf': 0.75, 'min_sample...",...,NaN,NaN,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
tree_model = tree_random.best_estimator_

In [ ]:
# Create parameter grid for AdaBoostRegressor cross validation
param = {'learning_rate': [0.001, 0.01, 0.05, 0.1, 0.3, 0.5, 1],
         'base_estimator': [tree_model],
         'n_estimators': [5, 10, 15, 20, 25, 50, 75, 100]}

In [ ]:
# AdaBoost cross validation
ada = AdaBoostRegressor()

ada_random = RandomizedSearchCV(estimator = ada, param_distributions = param, n_iter = 4, cv = cv, verbose=2, random_state=42, n_jobs = -1, return_train_score = True)

ada_random.fit(x_train, y_train)

Fitting 5 folds for each of 4 candidates, totalling 20 fits


C:\Users\jessl\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  20 out of  20 | elapsed:    5.1s finished


RandomizedSearchCV(cv=StratifiedKFold(n_splits=5, random_state=0, shuffle=True),
                   error_score=nan,
                   estimator=AdaBoostRegressor(base_estimator=None,
                                               learning_rate=1.0, loss='linear',
                                               n_estimators=50,
                                               random_state=None),
                   iid='deprecated', n_iter=4, n_jobs=-1,
                   param_distributions={'base_estimator': [DecisionTreeRegressor(ccp_alpha=0.0,
                                                                                 criterion='mse',
                                                                                 max_depth=20,
                                                                                 m...
                                                                                 min_impurity_decrease=0.0,
                                                              

In [ ]:
# View results in order of performance
ada_df = pd.DataFrame(ada_random.cv_results_).sort_values('mean_test_score', ascending = False)
ada_df.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,param_learning_rate,param_base_estimator,params,split0_test_score,split1_test_score,...,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,split3_train_score,split4_train_score,mean_train_score,std_train_score
3,1.485868,0.223235,0.015084,0.002164,50,0.01,"DecisionTreeRegressor(ccp_alpha=0.0, criterion...","{'n_estimators': 50, 'learning_rate': 0.01, 'b...",0.755670,0.592382,...,0.689004,0.061273,1,0.946756,0.952022,0.940590,0.950024,0.939583,0.945795,0.004965
1,1.470771,0.034940,0.016605,0.000806,50,0.001,"DecisionTreeRegressor(ccp_alpha=0.0, criterion...","{'n_estimators': 50, 'learning_rate': 0.001, '...",0.746005,0.596085,...,0.683578,0.065780,2,0.924013,0.940015,0.924037,0.934099,0.932529,0.930939,0.006172
2,0.296618,0.016286,0.005402,0.001027,10,0.3,"DecisionTreeRegressor(ccp_alpha=0.0, criterion...","{'n_estimators': 10, 'learning_rate': 0.3, 'ba...",0.702657,0.545230,...,0.667055,0.065273,3,0.962489,0.969640,0.961043,0.966433,0.963240,0.964569,0.003089
0,0.170341,0.016594,0.007493,0.005476,5,0.001,"DecisionTreeRegressor(ccp_alpha=0.0, criterion...","{'n_estimators': 5, 'learning_rate': 0.001, 'b...",0.690852,0.556154,...,0.631229,0.052830,4,0.900298,0.872070,0.886742,0.883562,0.902902,0.889115,0.011333


In [ ]:
ada_model = ada_random.best_estimator_
ada_model.fit(x_train,y_train)

y_pred = ada_model.predict(x_train)

In [ ]:
test_pred = ada_model.predict(x_test)
print(test_pred)

[ 6154474.33333333  2462902.25        7458766.75       ...
 47630758.25        4260191.71428571  1829233.6       ]


In [ ]:
# Try logistic regression
log = LogisticRegression()
log.fit(x_train,y_train)
test_pred = log.predict(x_test)

In [ ]:
# Try K-Neighbors Regression
model_knn = KNeighborsRegressor(n_neighbors=6)
model_knn.fit(x_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
                    metric_params=None, n_jobs=None, n_neighbors=6, p=2,
                    weights='uniform')

In [ ]:
test_pred = model_knn.predict(x_test)

In [ ]:
# Try XGBoost regression
xg_reg = xgb.XGBRegressor(colsample_bytree = 0.3, learning_rate = 0.001,
                max_depth = 3, alpha = 10, n_estimators = 15000)

In [ ]:
xg_reg.fit(x_train, y_train)

XGBRegressor(alpha=10, base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.3, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.001, max_delta_step=0, max_depth=3,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=15000, n_jobs=4, num_parallel_tree=1,
             objective='reg:squarederror', random_state=0, reg_alpha=10,
             reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method='exact',
             validate_parameters=1, verbosity=None)

In [ ]:
test_pred = xg_reg.predict(x_test)

In [ ]:
# This chunk of code is run for every modeling attempt to create a valid Kaggle submission for RMLSE testing
test["revenue"] = test_pred
result = test[["id","revenue"]]
result.to_csv("ada_tree4.csv", index=False)
result.head()

,id,revenue
3000,3001,6.154474e+06
3001,3002,2.462902e+06
3002,3003,7.458767e+06
3003,3004,1.083717e+07
3004,3005,2.565521e+06


In [ ]:
# Fourth submission RMLSE: 2.49591
# Baseline 1 RMLSE met.

In [ ]:
# Following the development of the AdaBoost decision tree, some additional feature engineering was done to reach the second baseline of 2.0
# Creating the franchise variable - RMLSE: 2.48561
# Modifying the production variable - RMLSE: 2.46957
# Creating the tagline dummy - RMLSE: 2.53704
# Creating the production dummy - RMLSE: 2.49776
# Creating the homepage dummy - RMLSE: 2.54564
# Add additional CV values for AdaBoost tuning - RMLSE: 2.48674
# Add additional CV values for DecisionTreeRegressor tuning - RMLSE: 2.50032
# Create budget and keyword variable - RMLSE: 2.51089
# Create variable for runtime and try normalization - RMLSE: 2.35785
# Creating date variables - RMLSE: 2.31145
# Try with cross-validated logistic regression - RMLSE: 3.97993
# More runtime feature engineering - RMLSE: 2.30573
# Removing ID variable - RMLSE: 2.28516
# Removing keyword dummy - RMLSE: 2.27979